In [3]:
# --- IMPORTS ---
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# ==========================================
# LAB 9: Data Loading & Exploration
# ==========================================
print("--- LAB 9 STEPS ---")

# 1. Read CSV
# Make sure the filename matches exactly what is in your folder
df = pd.read_csv('crime_prediction_dataset (1).csv')

# 2. Print Top 5 rows
print("\nTop 5 rows:")
print(df.head())

# 3. Print Number of Rows and Columns
print(f"\nRows: {df.shape[0]}")
print(f"Columns: {df.shape[1]}")

# 4. Determine null columns
print("\nNull Values:")
print(df.isnull().sum())

# 5. Filling null values (Mode) - Lab requirement
# (Dataset is clean, but we strictly follow the lab step)
df = df.fillna(df.mode().iloc[0])
print("\nChecked/Filled null values.")

# 6. Print datatype of each column
print("\nDatatypes:")
print(df.dtypes)


# ==========================================
# LAB 10: Data Cleaning & Preprocessing
# ==========================================
print("\n--- LAB 10 STEPS ---")

# 1. Checking unique values
print("\nUnique Districts:", df['District'].unique())

# 2. Dropping unnecessary columns
# We drop 'Month' (time series not needed for general risk) 
# We drop 'Reported_Incidents' because it reveals the answer directly
df.drop(['Month', 'Reported_Incidents'], axis=1, inplace=True)
print("\nDropped 'Month' and 'Reported_Incidents'.")

# 3. CREATING TARGET VARIABLE (Classification)
# We convert the numeric 'Crime_Rate_Index' into categories: Low, Medium, High
def get_risk_level(rate):
    if rate > 60: return 'High Risk'
    elif rate > 30: return 'Medium Risk'
    else: return 'Low Risk'

df['Risk_Level'] = df['Crime_Rate_Index'].apply(get_risk_level)
print("\nCreated 'Risk_Level' target column.")

# 4. Splitting into X and Y
# X = All columns except target and the original numeric rate
X = df.drop(['Crime_Rate_Index', 'Risk_Level'], axis=1)
y = df['Risk_Level']

print("\nFeatures (X):", list(X.columns))
print("Target (y): Risk_Level")

# 5. Encoding Categorical Columns
# We encode 'District' so the model can understand it
encoders = {}
cat_columns = X.select_dtypes(['object']).columns

for col in cat_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    encoders[col] = le

print("\nEncoded 'District' column.")


# ==========================================
# LAB 11: Model Training & Evaluation
# ==========================================
print("\n--- LAB 11 STEPS ---")

# 1. Splitting Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Applying Classifiers
# --- Model A: Random Forest ---
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# --- Model B: Naive Bayes (Lab requirement) ---
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)

# 3. Metrics
print("\n--- Random Forest Accuracy ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)*100:.4f}")

print("\n--- Naive Bayes Accuracy ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_nb)*100:.4f}")

# ==========================================
# SAVE FOR FLASK (LAB 12)
# ==========================================
# We save the Random Forest model as it usually performs better
data_to_save = {
    "model": rf_model,
    "encoders": encoders
}
with open('crime_model.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)
    
print("\nSUCCESS: Model saved as 'crime_model.pkl'")

--- LAB 9 STEPS ---

Top 5 rows:
     Month     District  Population  Unemployment_Rate  Median_Income  \
0  2024-01     Downtown       45000                5.8          42000   
1  2024-01  North Hills       82000                3.2          85000   
2  2024-01     Westside       65000                4.5          58000   
3  2024-01     East End       55000                7.2          35000   
4  2024-01      Suburbs      120000                2.8          92000   

   Police_Stations  Patrol_Hours_Daily  Reported_Incidents  Crime_Rate_Index  
0                4                 120                 345              76.6  
1                2                  45                 120              14.6  
2                3                  80                 210              32.3  
3                5                 140                 410              74.5  
4                3                  60                  95               7.9  

Rows: 80
Columns: 9

Null Values:
Month              